In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
from random import randrange

In [69]:

raw_shelves_df = pd.read_csv('data/shelves.csv')
raw_shedules_df = pd.read_csv('data/shedules.csv')
raw_products_df = pd.read_csv('data/products.csv')


In [70]:
shedules_df = raw_shedules_df.rename(columns={
    'N': 'id',
    'name_TT': 'store',
    'НаименованиеГрафика': 'schedule_name',
    'Длительность (недель)': 'duration_weeks',
    'Стеллажи': 'shelves',
    'неделя 1': 'week_1',
    'неделя 2': 'week_2',
    'неделя 3': 'week_3',
    'неделя 4': 'week_4',
    'Кол-во дней': 'days_count',
    'ЧастотаПостеллажная': 'shelf_frequency',
    'ЧастотаГрафика': 'schedule_frequency'
}, inplace=False)

shedules_df = shedules_df[shedules_df['store'].str.contains("5018ДС_СПб_Полюстровский59", case=False, na=False)]

shedules_df = shedules_df.melt(
    id_vars=['id', 'store', 'schedule_name', 'duration_weeks', 'shelves', 'days_count', 'shelf_frequency', 'schedule_frequency'],
    value_vars=['week_1', 'week_2', 'week_3', 'week_4'],
    var_name='week_number',
    value_name='week_day'
)

shedules_df = shedules_df.dropna(subset=['week_day']).reset_index(drop=True)

day_of_week_map = {'пн': 0, 'вт': 1, 'ср': 2, 'чт': 3, 'пт': 4, 'сб': 5, 'вс': 6}

shedules_df['day_index'] = shedules_df['week_day'].map(day_of_week_map)
shedules_df['day'] = shedules_df.apply(
    lambda row: (int(row['week_number'][-1]) - 1) * 7 + row['day_index'], axis=1
)


shedules_df = shedules_df.drop(columns=['week_number', 'week_day', 'day_index', 'schedule_frequency', 'shelf_frequency', 'days_count', 'duration_weeks'])
shedules_df = shedules_df.sort_values('day').reset_index(drop=True)

shedules_df.head()

,id,store,schedule_name,shelves,day
0,5018,5018ДС_СПб_Полюстровский59,Индивидуальный график постеллажной. ДС 5018,A1;A10;A10;A10;A10;A10;A10;A10;A10;A11;A11;A11...,0
1,5018,5018ДС_СПб_Полюстровский59,Индивидуальный график постеллажной. ДС 5018,K1;K10;K11;K12;K13;K14;K15;K16;K17;K18;K19;K2;...,1
2,5018,5018ДС_СПб_Полюстровский59,Индивидуальный график постеллажной. ДС 5018,G13;O1;O2;O3;O4;O5;O6;O7;O8;Г1;Г10;Г11;Г12;Г2;...,2
3,5018,5018ДС_СПб_Полюстровский59,Индивидуальный график постеллажной. ДС 5018,Z1;Z10;Z12;Z13;Z14;Z15;Z16;Z17;Z18;Z2;Z3;Z4;Z5...,3
4,5018,5018ДС_СПб_Полюстровский59,Индивидуальный график постеллажной. ДС 5018,Z18;Z19;Z20;Z21;Z22;Z23;Z24;Z25;Z26;Z27;Z28;Z2...,4


In [71]:
products_df = raw_products_df.rename(columns={
    'id_tov': 'id_tov',
    'Наименование': 'name',
    'Категория': 'category',
    'Группа': 'group',
    'Кол-во чеков за месяц': 'total_carts',
    'Единица измерения': 'system',
    'Кол-во за месяц': 'per_month',
    'Среднее кол-во за месяц': 'avg_per_month',
    'Сумма продаж за месяц': 'sum_per_month',
}, inplace=False)

# products_df = products_df[products_df['category'].str.contains("Овощи. Фрукты. Грибы. Зелень", case=False, na=False)]

products_df = products_df.reset_index(drop=True, inplace=False)

def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False
    

# Apply the filter to keep only rows where 'per_month' can be converted to float
products_df = products_df[products_df['per_month'].apply(is_float)]

# Convert the 'per_month' column to float after filtering
products_df['per_month'] = products_df['per_month'].astype(float)

products_df.head()

,id_tov,name,category,group,total_carts,system,per_month,avg_per_month,sum_per_month
0,23 074,Манго Египет,Овощи. Фрукты. Грибы. Зелень,Манго (все виды),8352,кг,6972.48,0.83,2194135.41
1,731,Бананы,Овощи. Фрукты. Грибы. Зелень,Банан,6869,кг,6338.46,0.92,952972.7
2,647,"Пакет-майка ""ВкусВилл"" малый",Нон-Фуд,"Сумки, пакеты, упаковка",87843,шт,96923.00,1.10,756861.36
3,19 419,Бедро цыпленка-бройлера бескостное (филе бедра),Мясо. Птица. П/ф охл. Яйца,Курица бескостное мясо,1504,кг,1438.22,0.96,723763.71
4,669,Арбуз,Овощи. Фрукты. Грибы. Зелень,Бахчевые,1621,кг,11265.38,6.95,683772.7


In [72]:

shelves_df = raw_shelves_df
shelves_df['shelve'] = shelves_df['shelve'].apply(lambda x: ''.join(x.split('-')[:2]))

shelves_df.head()

,name,price,shelve
0,Апельсины,215,A7
1,Апельсины Навелин,289,A3
2,Арбуз,67,A7
3,Баклажаны грунтовые,138,Fr1
4,Бананы,158,A7


In [73]:
def determine_trigger(sku_id, name, amount, avg_amount_in_cart, day_index):
  shelf_df = shelves_df[shelves_df['name'] == name]['shelve']
  shelf_df = shelf_df.reset_index(drop=True)
  
  day_df = shedules_df[shedules_df['day'] == day_index]
  day_df = day_df.reset_index(drop=True)
  
  if (not shelf_df.any() or not day_df['shelves'].any() or amount < avg_amount_in_cart): 
    return False
  
  return shelf_df[0] in day_df['shelves'][0]

In [74]:
# Now, for every product, for every day of the week, get the fucking trigger and sku_id

training_data = []
  
for _, row in products_df.iterrows():
  sku_id = row['id_tov']
  name = row['name']
  avg_per_month = row['avg_per_month']
  amount = randrange((int(row['avg_per_month']) + 1) * 10)
  
  # Add 13 rows for each product with dt from 0 to 12
  for dt in range(14):
    training_row = {
        'sku_id': sku_id,
        'name': name,
        'category': row['category'],  # Arbitrary value from products_df
        'amount': amount,
        'avg_cart': avg_per_month,
        't_expiery': 100,  # Arbitrary value
        'stocktaking_time': 30,  # Arbitrary value
        'dt': dt,
        'trigger': determine_trigger(sku_id, name, amount, avg_per_month, dt)
    }
    training_data.append(training_row)

training_df = pd.DataFrame(training_data)

training_df.to_csv('training_data.csv', index=False)


products_df.head(20)

,id_tov,name,category,group,total_carts,system,per_month,avg_per_month,sum_per_month
0,23 074,Манго Египет,Овощи. Фрукты. Грибы. Зелень,Манго (все виды),8352,кг,6972.48,0.83,2194135.41
1,731,Бананы,Овощи. Фрукты. Грибы. Зелень,Банан,6869,кг,6338.46,0.92,952972.7
2,647,"Пакет-майка ""ВкусВилл"" малый",Нон-Фуд,"Сумки, пакеты, упаковка",87843,шт,96923.00,1.10,756861.36
3,19 419,Бедро цыпленка-бройлера бескостное (филе бедра),Мясо. Птица. П/ф охл. Яйца,Курица бескостное мясо,1504,кг,1438.22,0.96,723763.71
4,669,Арбуз,Овощи. Фрукты. Грибы. Зелень,Бахчевые,1621,кг,11265.38,6.95,683772.7
5,488,Филе грудки цыпленка-бройлера,Мясо. Птица. П/ф охл. Яйца,Курица бескостное мясо,1314,кг,1241.24,0.94,616319.03
6,22 658,Яйцо куриное С0,Мясо. Птица. П/ф охл. Яйца,Яйца,2790,упак,3147.00,1.13,485488
7,16 306,"Масло сливочное 82,5%, 200 г",Молочная Продукция,Масло,1497,шт,1714.00,1.14,425423.71
8,36 296,"Молоко 2,5% в бутылке, 900 мл",Молочная Продукция,Молоко пастеризованное,3967,шт,5000.00,1.26,407427.92
9,64 336,Картофель молодой,Овощи. Фрукты. Грибы. Зелень,Картофель подгруппа,3508,кг,5922.67,1.69,389319.41
